In this approach we will use article title and calculate TF-IDF<br>

In [1]:
import pandas as pd

df = pd.read_csv('../dataset/corpus_with_parties_NE_24.10.2022.csv')
df = df.iloc[: , 3:]

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
df['text'] = df['text'].fillna('')
tfidf_matrix = tfidf.fit_transform(df['text'])

tfidf_matrix.shape

(2964, 45635)

In [3]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.25934393, 0.2659679 , ..., 0.09162891, 0.18282889,
        0.17750835],
       [0.25934393, 1.        , 0.16459576, ..., 0.06498178, 0.12689686,
        0.1250026 ],
       [0.2659679 , 0.16459576, 1.        , ..., 0.09665606, 0.14867722,
        0.16977711],
       ...,
       [0.09162891, 0.06498178, 0.09665606, ..., 1.        , 0.14474802,
        0.08423845],
       [0.18282889, 0.12689686, 0.14867722, ..., 0.14474802, 1.        ,
        0.14936205],
       [0.17750835, 0.1250026 , 0.16977711, ..., 0.08423845, 0.14936205,
        1.        ]])

In [4]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices.head(5)

title
Obama, Biden rally Democrats at annual retreat                                    0
Republicans call Obama’s budget a reelection ‘gimmick’                            1
Republicans aim to turn congressional wins into seats after redistricting         2
Illinois Democrats push out a Republican House incumbent                          3
Paul Ryan budget proposal sparks criticism from Democrats and some Republicans    4
dtype: int64

In [5]:
def content_recommender(title, cosine_sim, df, indices):

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    article_indices = [i[0] for i in sim_scores]
    
    return df[['title','parties']].iloc[article_indices]

In [6]:
#Republiacn
content_recommender('Republicans aim to turn congressional wins into seats after redistricting', cosine_sim, df, indices)

,title,parties
1762,Why Democrats can’t just obstruct their way ba...,['Democrats']
454,Republicans bet the House on staying on offense,['Republicans']
1543,The Republicans’ big gerrymander could backfir...,['Republicans']
906,Democrats have a depth problem,['Democrats']
272,Republicans poised to maintain control of the ...,['Republicans']
985,Which election was worse for Democrats: 2010 o...,['Democrats']
880,The one thing that unites Republicans? Calls f...,['Republicans']
2930,California just proved how cracking down on ge...,[]
2752,The 4 most severely undersold political storie...,[]
352,Democrats partner with political newcomers aim...,['Democrats']


In [7]:
#Democrats
content_recommender('Obama, Biden rally Democrats at annual retreat', cosine_sim, df, indices)

,title,parties
880,The one thing that unites Republicans? Calls f...,['Republicans']
110,Some Democrats worry about whether Obama’s cam...,['Democrats']
1522,"The Daily 202: Democrats claim patriotism, God...",['Democrats']
121,Will Obama fade or fight as Republicans gain c...,['Republicans']
2057,The Daily 202: The South is ready to save Hill...,[]
504,The ‘fiscal cliff’s’ Most Valuable Republicans,['Republicans']
1926,The Daily 202: Senate Republicans want to get ...,['Republicans']
1330,Obama gives pep talk to House Democrats,['Democrats']
1351,The Daily 202: Why blocking Obama’s pick to re...,['Republicans']
70,Obama seeks to defuse tensions among Democrats,['Democrats']
